In [ ]:
import numpy as np

def differential_evolution(cost_func, bounds, pop_size=20, mutation=0.8, crossover=0.7, generations=200):
    dimensions = len(bounds)
    pop = np.random.rand(pop_size, dimensions)
    min_b, max_b = np.asarray(bounds).T
    print(f"""
    Min b = {min_b}
    Max b = {max_b}
    """)
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([cost_func(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    
    for i in range(generations):
        for j in range(pop_size):
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + mutation * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossover
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = cost_func(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
    return best, fitness[best_idx]


In [23]:
def cost_function(params):
    # Unpack parameters
    Nx, Ny, Lx, Ly, h, Nr, Lr, A_grid, t_rev = params

    # Material costs (per unit)
    c_conductor = 0  # $/m
    c_excavation = 261.59 # $/m³
    c_welding = 12  # $/joint
    c_rod = 0  # $/m
    c_rev = 25  # $/m³

    # Cost components
    C1 = (Nx * Lx + Ny * Ly) * c_conductor
    C2 = (Nx * Lx + Ny * Ly) * h * c_excavation
    C3 = (Nx + Ny) * c_welding
    C4 = Nr * Lr * c_rod
    C5 = A_grid * t_rev * c_rev
    C6 = A_grid * h * c_excavation

    # Total cost
    C_total = C1 + C2 + C3 + C4 + C5 + C6
    return C_total


In [24]:
bounds = [
    (10, 150),        # Nx: No. of conductors in x-direction
    (10, 150),        # Ny: No. of conductors in y-direction
    (10.0, 200.0),    # Lx: Length of each x-conductor (m)
    (10.0, 200.0),    # Ly: Length of each y-conductor (m)
    (0.5, 2.5),       # h: Depth of burial (m) – deeper for 345 kV systems
    (10, 200),        # Nr: No. of vertical rods
    (1.0, 15.0),      # Lr: Length of each rod (m)
    (161874, 161874.0),# A_grid: Area of the grounding grid (m²)
    (0.05, 0.3)       # t_rev: Thickness of revetment layer (m)
]

In [25]:
best_params, min_cost = differential_evolution(cost_function, bounds)
print("Optimal Parameters:", best_params)
print("Minimum Cost: $", min_cost)

Optimal Parameters: [1.00000000e+01 1.00000000e+01 1.00000000e+01 1.00000000e+01
 5.00000000e-01 2.00000000e+02 1.03136228e+01 1.61874000e+05
 5.00000000e-02]
Minimum Cost: $ 21401051.33
